In [1]:
!pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 390.1 kB/s eta 0:00:00a 0:00:01


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("ETL_SmartFarming") \
    .getOrCreate()

In [2]:
from pyspark.sql.functions import regexp_extract, to_timestamp, col
from pyspark.sql.types import IntegerType
import os

# HDFS path
hdfs_input_path = "hdfs://namenode:9000/user/smart_farming_data/*"

#to track last processed timestamp
CHECKPOINT_FILE = "/tmp/last_processed_timestamp.txt"

def get_last_processed_timestamp():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            return f.read().strip()
    return None

#savingg last processed timestamp
def save_last_processed_timestamp(timestamp):
    with open(CHECKPOINT_FILE, 'w') as f:
        f.write(str(timestamp))

In [3]:
#reading data from hadoop
df = spark.read.parquet(hdfs_input_path)
df = df.withColumn("timestamp", to_timestamp("timestamp"))
df = df.withColumn("farm_id", regexp_extract("farm_id", r"(\d+)", 1).cast(IntegerType()))

#filtering to get new records only
last_timestamp = get_last_processed_timestamp()

if last_timestamp:
    print(f"Last processed: {last_timestamp}")
    print(f"Filtering for new records after {last_timestamp}...")
    df = df.filter(col("timestamp") > last_timestamp)
    new_count = df.count()
    print(f"found {new_count:,} new records")
    
    if new_count == 0:
        print("No new data. Exiting.") #Iadded this line to avoid the code crashing if there's no new data
        spark.stop()
        raise SystemExit("No new data to process")  
else:
    print("First run - processing all data")

df.cache()
max_timestamp = df.agg({"timestamp": "max"}).collect()[0][0]

First run - processing all data


In [4]:
df.printSchema()
df.show(30)

root
 |-- sensor_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- soil_moisture: double (nullable = true)
 |-- soil_pH: double (nullable = true)
 |-- temperature: double (nullable = true)
 |-- rainfall: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- sunlight_intensity: double (nullable = true)
 |-- pesticide_usage_ml: double (nullable = true)
 |-- farm_id: integer (nullable = true)
 |-- region: string (nullable = true)
 |-- crop_type: string (nullable = true)

+--------------------+--------------------+-------------+-------+-----------+--------+--------+------------------+------------------+-------+----------+---------+
|           sensor_id|           timestamp|soil_moisture|soil_pH|temperature|rainfall|humidity|sunlight_intensity|pesticide_usage_ml|farm_id|    region|crop_type|
+--------------------+--------------------+-------------+-------+-----------+--------+--------+------------------+------------------+-------+----------+-----

In [5]:
#handling missing values
import pyspark.sql.functions as F
numeric_cols = ["soil_moisture", "soil_pH", "temperature", "humidity", "sunlight_intensity"]
mean_exprs = [F.mean(c).alias(c) for c in numeric_cols]
means = df.select(mean_exprs).collect()[0] 
fill_dict = {c: means[c] for c in numeric_cols}
df = df.fillna(fill_dict)


In [6]:
#cleaning outliers
from pyspark.sql import functions as F

numeric_cols_outliers = [
    "soil_moisture",
    "soil_pH", 
    "temperature",
    "humidity",
    "sunlight_intensity"
]

bounds = {}
for col_name in numeric_cols_outliers:
    Q1, Q3 = df.approxQuantile(col_name, [0.25, 0.75], 0.01) 
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR 
    upper = Q3 + 1.5 * IQR
    bounds[col_name] = (lower, upper)
    print(f"{col_name}: Range=[{lower:.2f}, {upper:.2f}]")

filter_conditions = [
    (F.col(col_name).between(bounds[col_name][0], bounds[col_name][1])) 
    for col_name in numeric_cols_outliers
]
combined_filter = filter_conditions[0]
for condition in filter_conditions[1:]:
    combined_filter = combined_filter & condition

total_rows = df.count()
df_cleaned = df.filter(combined_filter)
rows_after = df_cleaned.count()

print(f"\nRows before: {total_rows}")
print(f"Rows after: {rows_after}")
print(f"Rows removed: {total_rows - rows_after} ({((total_rows - rows_after) / total_rows * 100):.2f}%)")

df = df_cleaned  

soil_moisture: Range=[-17.84, 127.76]
soil_pH: Range=[4.41, 9.57]
temperature: Range=[14.40, 15.92]
humidity: Range=[52.64, 64.92]
sunlight_intensity: Range=[0.00, 0.00]

Rows before: 1537
Rows after: 1458
Rows removed: 79 (5.14%)


In [7]:
from pyspark.sql.functions import trim, col
#removing spaces
text_cols = ["region", "crop_type"]
for c in text_cols:
    df = df.withColumn(c, trim(col(c)))
#showing resultss
df.show(5)

+--------------------+--------------------+-------------+-------+-----------+--------+--------+------------------+------------------+-------+---------+---------+
|           sensor_id|           timestamp|soil_moisture|soil_pH|temperature|rainfall|humidity|sunlight_intensity|pesticide_usage_ml|farm_id|   region|crop_type|
+--------------------+--------------------+-------------+-------+-----------+--------+--------+------------------+------------------+-------+---------+---------+
|1bc6f015-5535-49d...|2025-11-26 22:10:...|        39.35|    6.4|      15.68|     0.0|   58.19|               0.0|               0.0|      1|NileDelta|    Wheat|
|5598dfe2-3013-451...|2025-11-26 22:11:...|        34.36|   6.73|      15.53|     0.0|   58.26|               0.0|               0.0|      3|NileDelta|    Onion|
|32619bc8-acf0-478...|2025-11-26 22:11:...|        39.42|   6.39|       15.6|     0.0|   61.05|               0.0|               0.0|      1|NileDelta|    Wheat|
|5ee9047d-c410-44b...|2025-1

In [8]:
from pyspark.sql.functions import col, year, month, dayofmonth, weekofyear, hour, to_date, minute
#extracting date parts
df = df.withColumn("date", to_date("timestamp")) \
       .withColumn("year", year("timestamp")) \
       .withColumn("month", month("timestamp")) \
       .withColumn("day", dayofmonth("timestamp")) \
       .withColumn("week", weekofyear("timestamp")) \
       .withColumn("hour", hour("timestamp")) \
       .withColumn("minute", minute("timestamp")) 

In [9]:
for c in numeric_cols:
    df = df.withColumn(c, F.col(c).cast("double"))

In [10]:
df.show(20)

+--------------------+--------------------+-------------+-------+-----------+--------+--------+------------------+------------------+-------+----------+---------+----------+----+-----+---+----+----+------+
|           sensor_id|           timestamp|soil_moisture|soil_pH|temperature|rainfall|humidity|sunlight_intensity|pesticide_usage_ml|farm_id|    region|crop_type|      date|year|month|day|week|hour|minute|
+--------------------+--------------------+-------------+-------+-----------+--------+--------+------------------+------------------+-------+----------+---------+----------+----+-----+---+----+----+------+
|1bc6f015-5535-49d...|2025-11-26 22:10:...|        39.35|    6.4|      15.68|     0.0|   58.19|               0.0|               0.0|      1| NileDelta|    Wheat|2025-11-26|2025|   11| 26|  48|  22|    10|
|5598dfe2-3013-451...|2025-11-26 22:11:...|        34.36|   6.73|      15.53|     0.0|   58.26|               0.0|               0.0|      3| NileDelta|    Onion|2025-11-26|202

In [11]:
#Moisture trend over time, we calculate the average soil moisture for each farm and region per day
moisture_trend = df.groupBy("date", "region", "farm_id") \
    .agg(F.avg("soil_moisture").alias("avg_soil_moisture"))

In [12]:
#rainfall vs soil moisture, we compare average rainfall with average soil moisture per region per day
rain_moisture = df.groupBy("date", "region") \
    .agg(F.sum("rainfall").alias("total_rainfall"),
         F.avg("soil_moisture").alias("avg_soil_moisture"))

In [13]:
#Effect of climate on soil moisture, we observe how temperature and sunlight affect soil moisture per region per day
climate_effect = df.groupBy("date", "region") \
    .agg(F.avg("soil_moisture").alias("avg_soil_moisture"),
         F.avg("temperature").alias("avg_temperature"),
         F.avg("sunlight_intensity").alias("avg_sunlight"))


In [14]:
#Soil pH trend, we track average soil pH over time for each crop and region
ph_trend = df.groupBy("date", "region", "crop_type") \
    .agg(F.avg("soil_pH").alias("avg_soil_pH"))


In [15]:
#Rainfall vs pesticide usage, we monitor rainfall impact on pesticide application
rain_pesticide = df.groupBy("date", "region") \
    .agg(F.sum("rainfall").alias("total_rainfall"),
         F.sum("pesticide_usage_ml").alias("total_pesticide_usage"))


In [16]:
#Sunlight exposure, we get the sum daily sunlight hours per region
sunlight_daily = df.groupBy("date", "region") \
    .agg(F.sum("sunlight_intensity").alias("total_sunlight_hours"))

In [17]:
#Pesticide usage trend, we track total pesticide usage per crop and region over time
pesticide_trend = df.groupBy("date", "region", "crop_type") \
    .agg(F.sum("pesticide_usage_ml").alias("total_pesticide_usage"))

In [18]:
df.show(20)

+--------------------+--------------------+-------------+-------+-----------+--------+--------+------------------+------------------+-------+----------+---------+----------+----+-----+---+----+----+------+
|           sensor_id|           timestamp|soil_moisture|soil_pH|temperature|rainfall|humidity|sunlight_intensity|pesticide_usage_ml|farm_id|    region|crop_type|      date|year|month|day|week|hour|minute|
+--------------------+--------------------+-------------+-------+-----------+--------+--------+------------------+------------------+-------+----------+---------+----------+----+-----+---+----+----+------+
|1bc6f015-5535-49d...|2025-11-26 22:10:...|        39.35|    6.4|      15.68|     0.0|   58.19|               0.0|               0.0|      1| NileDelta|    Wheat|2025-11-26|2025|   11| 26|  48|  22|    10|
|5598dfe2-3013-451...|2025-11-26 22:11:...|        34.36|   6.73|      15.53|     0.0|   58.26|               0.0|               0.0|      3| NileDelta|    Onion|2025-11-26|202

In [19]:
from pyspark.sql.functions import col, trim

dim_farm = df.select(
    col("farm_id").cast("int").alias("farm_id"),
    trim(col("region")).alias("region")
).dropDuplicates(["farm_id"])

dim_crop = df.select(
    col("crop_type").substr(1,50).alias("crop_type")
).dropDuplicates(["crop_type"])

dim_time = df.select(
    col("date").cast("date").alias("date"),
    col("year").cast("int").alias("year"),
    col("month").cast("int").alias("month"),
    col("day").cast("int").alias("day"),
    col("week").cast("int").alias("week"),
    col("hour").cast("int").alias("hour"),
    col("minute").cast("int").alias("minute")
).dropDuplicates(["date", "hour", "minute"])


fact_sensor_data = df.select(
    col("sensor_id").substr(1,50).alias("sensor_id"),
    col("timestamp").alias("timestamp"),
    col("soil_moisture").cast("double").alias("soil_moisture"),
    col("soil_pH").cast("double").alias("soil_pH"),
    col("temperature").cast("double").alias("temperature"),
    col("rainfall").cast("double").alias("rainfall"),
    col("humidity").cast("double").alias("humidity"),
    col("sunlight_intensity").cast("double").alias("sunlight_intensity"),
    col("pesticide_usage_ml").cast("double").alias("pesticide_usage_ml"),
    col("farm_id").cast("int").alias("farm_id"),  
    col("crop_type").substr(1,50).alias("crop_type"), 
    col("date").cast("date").alias("date"),    
    col("hour").cast("int").alias("hour"),
    col("minute").cast("int").alias("minute")
).dropDuplicates(["sensor_id", "timestamp"])


In [20]:
import pymysql
from sqlalchemy import create_engine

url = "jdbc:mysql://mysql:3306/farm_dwh"
properties = {"user": "root", "password": "root", "driver": "com.mysql.cj.jdbc.Driver"}


tables_to_write = {
    "dim_farm": (dim_farm, "farm_id INT, region VARCHAR(100)"),
    "dim_crop": (dim_crop, "crop_type VARCHAR(50)"),
    "dim_time": (dim_time, "date DATE, year INT, month INT, day INT, week INT, hour INT, minute INT"),    
    "fact_sensor_data": (fact_sensor_data, """
        sensor_id VARCHAR(50),
        timestamp TIMESTAMP,
        soil_moisture DOUBLE,
        soil_pH DOUBLE,
        temperature DOUBLE,
        rainfall DOUBLE,
        humidity DOUBLE,
        sunlight_intensity DOUBLE,
        pesticide_usage_ml DOUBLE,
        farm_id INT,
        crop_type VARCHAR(50),
        date DATE,
        hour INT,
        minute INT
    """),
    "moisture_trend": (moisture_trend, "date DATE, region VARCHAR(100), farm_id INT, avg_soil_moisture DOUBLE"),
    "rain_moisture": (rain_moisture, "date DATE, region VARCHAR(100), total_rainfall DOUBLE, avg_soil_moisture DOUBLE"),
    "climate_effect": (climate_effect, "date DATE, region VARCHAR(100), avg_soil_moisture DOUBLE, avg_temperature DOUBLE, avg_sunlight DOUBLE"),
    "ph_trend": (ph_trend, "date DATE, region VARCHAR(100), crop_type VARCHAR(50), avg_soil_pH DOUBLE"),
    "rain_pesticide": (rain_pesticide, "date DATE, region VARCHAR(100), total_rainfall DOUBLE, total_pesticide_usage DOUBLE"),
    "sunlight_daily": (sunlight_daily, "date DATE, region VARCHAR(100), total_sunlight_hours DOUBLE"),
    "pesticide_trend": (pesticide_trend, "date DATE, region VARCHAR(100), crop_type VARCHAR(50), total_pesticide_usage DOUBLE")
}
fact_sensor_data = fact_sensor_data.orderBy("timestamp").coalesce(1)

for table_name, (df, col_types) in tables_to_write.items():
    print(f"Loading data into {table_name}")
    df.write.jdbc(
        url=url,
        table=table_name,
        mode="append",
        properties={**properties, "createTableColumnTypes": col_types}
    )
    print(f"Successfully loaded {table_name}\n")

print("All tables successfully loaded to MySQL")

Loading data into dim_farm
Successfully loaded dim_farm

Loading data into dim_crop
Successfully loaded dim_crop

Loading data into dim_time
Successfully loaded dim_time

Loading data into fact_sensor_data
Successfully loaded fact_sensor_data

Loading data into moisture_trend
Successfully loaded moisture_trend

Loading data into rain_moisture
Successfully loaded rain_moisture

Loading data into climate_effect
Successfully loaded climate_effect

Loading data into ph_trend
Successfully loaded ph_trend

Loading data into rain_pesticide
Successfully loaded rain_pesticide

Loading data into sunlight_daily
Successfully loaded sunlight_daily

Loading data into pesticide_trend
Successfully loaded pesticide_trend

All tables successfully loaded to MySQL


In [21]:
print(f"\n Saving checkpoint: {max_timestamp}")
save_last_processed_timestamp(max_timestamp)
print(f"✓ Checkpoint saved. Next run will process data after {max_timestamp}")


 Saving checkpoint: 2025-11-26 22:36:59.150555
✓ Checkpoint saved. Next run will process data after 2025-11-26 22:36:59.150555
